<a href="https://colab.research.google.com/github/andre-arantes/ia/blob/master/tp1/etapa5/K_Nearest_Neighbors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
with open('/content/sample_data/PreProcessamento.pkl', 'rb') as f:
  X_trainscaled, X_testscaled, y_treino, y_teste = pickle.load(f)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report

In [ ]:
#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
model = KNeighborsClassifier()

In [ ]:
from sklearn.metrics import make_scorer, recall_score

# Create a custom scoring function for recall of class 1
scoring = {'Recall_Class1': make_scorer(recall_score, pos_label=1)}

# Random search of parameters, using 5 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=model, param_distributions=hyperparameters,
                              n_iter = 100, scoring=scoring,refit='Recall_Class1',
                              cv = 5, verbose=2, random_state=0, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(X_trainscaled, y_treino)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'leaf_size': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30, ...],
                                        'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8,
                                                        9, 10, 11, 12, 13, 14,
                                                        15, 16, 17, 18, 19, 20,
                                                        21, 22, 23, 24, 25, 26,
                                                        27, 28, 29],
                                        'p': [1, 2]},
                   random_state=0, refit='Recall_Class1',
                   return_train_score=True,
                   scoring={'Recall_Class1': make_scorer(recall_score, pos_label=1)},
                   verbose=2)

**testando o modelo**

In [ ]:
best_model = rf_random.best_estimator_

In [ ]:
previsoes = best_model.predict(X_testscaled)

In [ ]:
print(classification_report(y_teste, previsoes))
report_Kmeans = classification_report(y_teste, previsoes, output_dict=True)
relatorio_Kmeans = report_Kmeans['1']['recall']

              precision    recall  f1-score   support

           0       0.99      0.92      0.95     17177
           1       0.32      0.76      0.45       901

    accuracy                           0.91     18078
   macro avg       0.65      0.84      0.70     18078
weighted avg       0.95      0.91      0.93     18078



**Serializar Classification Report**

In [ ]:
import pickle
with open('/content/sample_data/RelatorioKmeans.pkl', mode = 'wb') as f:
  pickle.dump([relatorio_Kmeans], f)